#Next Frame Prediction using Pix2PixHD

Notbook adapted from Derrick Schultz's Next Frame Prediction using Pix2PixHD which is Forked repo and tutorial based on [JC Testud’s excellent repo and Medium](https://medium.com/@jctestud/video-generation-with-pix2pix-aed5b1b69f57) article.

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Install libraries and dependencies



In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/nfp-colab/pix2pixHD/"):
    %cd /content/drive/MyDrive/nfp-colab/pix2pixHD/
    # !git pull
    !pip install dominate
else:
    %cd /content/drive/MyDrive
    !mkdir nfp-colab
    %cd nfp-colab
    !git clone -b video https://github.com/dvschultz/pix2pixHD.git
    !pip install dominate
    %cd pix2pixHD/

process video using ffmpeg

In [ ]:
# scale video
!ffmpeg -i /content/drive/MyDrive/dmlap-final/stylegan/stylegan3/trypophobia0-7.mp4 -vf "scale=1280:736" /content/drive/MyDrive/dmlap-final/Vids/source/trypophobia0-7resized.mp4.mp4

In [ ]:
# trim video to 30s
!ffmpeg -i /content/drive/MyDrive/dmlap-final/Vids/source/trypophobia0-7resized.mp4.mp4 -t 30 /content/drive/MyDrive/dmlap-final/Vids/source/trypophobia_30s.mp4

In [ ]:
# extract frames
!ffmpeg -ss 00:00:00 -i "/content/sample2.mp4" -vf "fps=24" "/content/drive/MyDrive/nfp-colab/pix2pixHD/datasets/londonBridge_trim6/test_frames/frame_%06d.png"

## Extract frames from video

Upload a video to either Colab or Google Drive.

* `-video` is the path to the video file
* `-name` should be a unique project name
* `-width` and `-height` **must** to be a multiple of 32
* `-p2pdir` leave as `.`


In [ ]:
!python extract_frames.py -video /content/drive/MyDrive/dmlap-final/Vids/source/trypophobia_30s.mp4 -name trypophobia -p2pdir . -width 1280 -height 736

creating the dataset structure
ffmpeg -v 16 -i /content/drive/MyDrive/dmlap-final/Vids/source/trypophobia_30s.mp4 -q:v 2 -vf "scale=iw*736/ih:736, crop=1280:736" /content/drive/MyDrive/nfp-colab/pix2pixHD/datasets/trypophobia/train_frames/frame-%06d.jpg -hide_banner
extracting the frames


## Train your model

### Initial training

Note: if you have a large dataset, this may timeout initially (`ValueError: __len__() should return >= 0`). Give it a minute or two and run it again.

*   `--name` should be a unique name (think of it like a project name). For your sanity I recommend using the same `-name` as above.
*   `--dataroot` should point to your dataset. This will usually be in `./datasets/[your project name]`





In [ ]:
# with schedule sampling
!python train_video.py --name water_project --dataroot ./datasets/water_dataset/ --save_epoch_freq 1 --ngf 64 --scheduled_sampling --serial_batches

In [ ]:
!python train_video.py --name fire_noss --dataroot ./datasets/fire/ --save_epoch_freq 1

trying to fix this error appears to be a version hell issue:


```
Traceback (most recent call last):
  File "/content/drive/MyDrive/nfp-colab/pix2pixHD/train_video.py", line 55, in <module>
    opt.print_freq = lcm(opt.print_freq, opt.batchSize)
  File "/content/drive/MyDrive/nfp-colab/pix2pixHD/train_video.py", line 11, in lcm
    def lcm(a,b): return abs(a * b)/fractions.gcd(a,b) if a and b else 0
AttributeError: module 'fractions' has no attribute 'gcd'
```

will this be a fix?
https://stackoverflow.com/questions/66174862/import-error-cant-import-name-gcd-from-fractions

(fixed by changing this in train_video.py


```
def lcm(a,b): return abs(a * b)/fractions.gcd(a,b) if a and b else 0
```

to

```
import math
def lcm(a,b): return abs(a * b)/math.gcd(a,b) if a and b else 0
```

now the training runs no problem!

### Continue Training
To pick up from a previous training session run the same command you ran to start with and append `--continue_train` to the end of the command.

In [ ]:
!python train_video.py --name londonBridge_ss --dataroot ./datasets/londonBridge_480/ --save_epoch_freq 2 --continue_train

#Generating Videos

There are a number of additional arguments you’ll need to include in this command:

*   `--fps` frame rate for your video
*   `--how_many` how many frames you want to create (this + fps = video length)
*   `--which_epoch` which epoch you want to generate videos from (note: if you choose `133` but there’s no epoch 133 model file, this will throw an error)
*   `--start_from` by default your video will start predicting images from the 60th frame of your training set. You can pass in the path to a different frame to have it start from that frame



In [ ]:
!python generate_video.py --name londonBridge_ss --dataroot ./datasets/firetrim2/ --fps 24 --how_many 800 --which_epoch latest --resize_or_crop none

In [ ]:
!python generate_progress_video.py --name fire_noss --dataroot ./datasets/fire/ --fps 24 --ngf 64 --pstart 1 --pstop 40

In [ ]:
# reverse video
!ffmpeg -i /content/12s.mp4 -vf reverse /content/12s-reversed.mp4

In [ ]:
# convert format
!ffmpeg -i "/content/drive/MyDrive/nfp-colab/pix2pixHD/hand.MOV" -c:v libx264 -c:a aac "/content/drive/MyDrive/nfp-colab/pix2pixHD/hand.mp4"

In [ ]:
# merge audio to video
!ffmpeg -i stitchedfin.mp4 -i londonBridge.mp4 -c:v copy -c:a aac -map 0:v:0 -map 1:a:0 hand-audio2.mp4